In [1]:
%%capture
!pip install datasets
!pip install streamlit
!npm install localtunnel
!pip install pyngrok
!pip install pdfplumber

In [4]:
%%writefile app.py

import streamlit as st
from transformers import  pipeline, LEDForConditionalGeneration, LEDTokenizer
import torch
import pdfplumber

model = LEDForConditionalGeneration.from_pretrained("silmi224/exp2-led-risalah_data_v2", return_dict_in_generate=True).to("cuda")
tokenizer = LEDTokenizer.from_pretrained("silmi224/exp2-led-risalah_data_v2")

def generate_summary(input_text, max_input, min_input):

    input_ids = tokenizer(input_text, truncation=True, return_tensors="pt").input_ids.to("cuda")
    global_attention_mask = torch.zeros_like(input_ids)

    # set global_attention_mask pada token pertama
    global_attention_mask[:, 0] = 1

    sequences = model.generate(input_ids, global_attention_mask=global_attention_mask, max_length=max_input, min_length=min_input).sequences

    generated_summary = tokenizer.decode(sequences[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return generated_summary

def extract_text_from_pdf(pdf_file):
    text = ""
    try:
        # ektraks text dengan pdfplumber
        with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                text += page_text
    except Exception as e:
        print(f"Error extracting text from {pdf_file}: {str(e)}")

    return text

# Streamlit app
# def main():
#     st.title("Peringkasan Teks Risalah Rapat dengan Longformer")

#     # Text input
#     input_text = st.text_area("Masukkan teks risalah rapat (Bahasa Indonesia):", height=300)

# #     max_input = st.slider("Panjang maksimum ringkasan yang diinginkan", min_value=50, max_value=512, value=1024, step=10)
# #     min_input = st.slider("Minimum length of the summary", min_value=30, max_value=512, value=50, step=10)

#     min_input=128
#     max_input=1024

#     if st.button("Generate Ringkasan"):
#         if input_text:
#             with st.spinner("Generating summary..."):
#                 summary = generate_summary(input_text, max_input, min_input)
#                 st.subheader("Hasil Ringkasan")
#                 st.write(summary)
#         else:
#             st.error("Please enter text to summarize.")

# if __name__ == "__main__":
#     main()

def main():
    st.title("Peringkasan Teks Risalah Rapat dengan Model Longformer")

    uploaded_file = st.file_uploader("Unggah file PDF risalah rapat:", type="pdf")

    if uploaded_file is not None:
        # Extract text from uploaded PDF file
        input_text = extract_text_from_pdf(uploaded_file)
        st.write("Teks yang diambil dari PDF:")
        st.write(input_text)
    else:
        # Display text area for manual input if no file is uploaded
        direct_input_text = st.text_area("Atau, masukkan teks risalah rapat secara langsung:", height=300)
        input_text = direct_input_text

    if input_text:
        max_input = 1024
        min_input = 50
        if st.button("Generate Summary"):
            with st.spinner("Generating summary..."):
                summary = generate_summary(input_text, max_input, min_input)
                st.subheader("Hasil Ringkasan")
                st.write(summary)
    else:
        st.info("Silakan unggah file PDF atau masukkan teks untuk diringkas.")

if __name__ == "__main__":
    main()


Overwriting app.py


# **LOCALTUNNEL**

In [4]:
!curl https://loca.lt/mytunnelpassword

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

35.203.145.10

100    13  100    13    0     0     30      0 --:--:-- --:--:-- --:--:--    30


0

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.203.145.10:8501

your url is: https://hungry-falcons-take.loca.lt


2024-08-22 03:21:28.170914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 03:21:28.170961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 03:21:28.172648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 03:21:36.462262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 03:21:36.462320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor

# **NGROK**

In [5]:
from pyngrok import ngrok
import os

get_ipython().system = os.system

# Untuk authentication token
ngrok.set_auth_token("2jsRlpbU1vY6CsS4EyRwhEbpcKR_3yQ1Ar8rtp9gpGTT8Zoe4")

# Start server streamlit
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok untuk mengekspos streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print url
print(' * Tunnel URL:', ngrok_tunnel.public_url)

 * Tunnel URL: https://f01e-35-204-168-90.ngrok-free.app




2024-08-25 16:52:26.949 Port 5011 is already in use


In [ ]:
!streamlit run app.py

In [ ]:
!ngrok authtoken $2jsRlpbU1vY6CsS4EyRwhEbpcKR_3yQ1Ar8rtp9gpGTT8Zoe4

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'


In [ ]:
!streamlit run app.py